In [48]:
import torch
import torch_geometric.transforms as T
import matplotlib.pyplot as plt
import collections
import numpy as np
import pandas as pd

In [149]:
from torch_geometric.datasets import Planetoid

data_Cora =Planetoid(root='/tmp/Cora', name='Cora',transform=T.NormalizeFeatures())[0]
#data_cora =dataset[0]
data_Citeseer =Planetoid(root='/tmp/Citeseer', name='Citeseer',transform=T.NormalizeFeatures())[0]
data_Pubmed =Planetoid(root='/tmp/Pubmed', name='Pubmed',transform=T.NormalizeFeatures())[0]

from torch_geometric.datasets import Amazon

data_Photo =Amazon(root='/tmp/Photo', name='Photo',transform=T.NormalizeFeatures())[0]
data_Computers =Amazon(root='/tmp/Computers', name='Computers',transform=T.NormalizeFeatures())[0]

from torch_geometric.datasets import Coauthor

data_CS =Coauthor(root='/tmp/CS', name='CS',transform=T.NormalizeFeatures())[0]
data_Physics =Coauthor(root='/tmp/Physics', name='Physics',transform=T.NormalizeFeatures())[0]

from torch_geometric.datasets import Flickr
from torch_geometric.datasets import Reddit


#social networks: Reddit, Flickr
data_Flickr =Flickr(root='/tmp/Flickr',transform=T.NormalizeFeatures())[0]
# data_Reddit = Reddit(root='/tmp/Reddit',transform=T.NormalizeFeatures()) очень тяжелый 

#Wikipedia "Chameleon", "Squirrel"

#Datasets of varying the homofily
from torch_geometric.datasets import MixHopSyntheticDataset
data_Homofily_0 = MixHopSyntheticDataset(root='/tmp/MixHopSyntheticDataset',homophily = 0.0, transform=T.NormalizeFeatures())[0]
data_Homofily_01 = MixHopSyntheticDataset(root='/tmp/MixHopSyntheticDataset',homophily = 0.1, transform=T.NormalizeFeatures())[0]
data_Homofily_02 = MixHopSyntheticDataset(root='/tmp/MixHopSyntheticDataset',homophily = 0.2, transform=T.NormalizeFeatures())[0]
data_Homofily_03 = MixHopSyntheticDataset(root='/tmp/MixHopSyntheticDataset',homophily = 0.3, transform=T.NormalizeFeatures())[0]
data_Homofily_04 = MixHopSyntheticDataset(root='/tmp/MixHopSyntheticDataset',homophily = 0.4, transform=T.NormalizeFeatures())[0]
data_Homofily_05 = MixHopSyntheticDataset(root='/tmp/MixHopSyntheticDataset',homophily = 0.5, transform=T.NormalizeFeatures())[0]
data_Homofily_06 = MixHopSyntheticDataset(root='/tmp/MixHopSyntheticDataset',homophily = 0.6, transform=T.NormalizeFeatures())[0]
data_Homofily_07 = MixHopSyntheticDataset(root='/tmp/MixHopSyntheticDataset',homophily = 0.7, transform=T.NormalizeFeatures())[0]
data_Homofily_08 = MixHopSyntheticDataset(root='/tmp/MixHopSyntheticDataset',homophily = 0.8, transform=T.NormalizeFeatures())[0]
data_Homofily_09 = MixHopSyntheticDataset(root='/tmp/MixHopSyntheticDataset',homophily = 0.9, transform=T.NormalizeFeatures())[0]

In [152]:
data_Homofily_01

Data(edge_index=[2, 59596], test_mask=[5000], train_mask=[5000], val_mask=[5000], x=[5000, 2], y=[5000])

In [154]:
datasets = [data_Cora, data_Citeseer, data_Pubmed, data_Photo, data_Computers, data_CS, data_Physics,data_Flickr,data_Homofily_0, data_Homofily_01,data_Homofily_02,data_Homofily_03,data_Homofily_04,data_Homofily_05,data_Homofily_06,data_Homofily_07,data_Homofily_08,data_Homofily_09]
datasets_names = ['Cora', 'Citeseer', 'Pubmed', 'Photo', 'Computers', 'CS', 'Physics','Flickr','Homophily0','Homophily01','Homophily02','Homophily03','Homophily04','Homophily05','Homophily06','Homophily07','Homophily08','Homophily09']

In [155]:

def to_networkx(data, node_attrs=None, edge_attrs=None):
    r"""Converts a data object graph to a networkx graph.

    Args:
        data (torch_geometric.data.Data): The data object.
        node_attrs (iterable of str, optional): The node attributes to be
            copied. (default: :obj:`None`)
        edge_attrs (iterable of str, optional): The edge attributes to be
            copied. (default: :obj:`None`)
    """

    G = nx.Graph()
    G.add_nodes_from(range(data.num_nodes))

    values = {key: data[key].squeeze().tolist() for key in data.keys}

    for i, (u, v) in enumerate(data.edge_index.t().tolist()):
        G.add_edge(u, v)
        for key in edge_attrs if edge_attrs is not None else []:
            G[u][v][key] = data[key][i]

    for key in node_attrs if node_attrs is not None else []:
        for i, feat_dict in G.nodes(data=True):
            feat_dict.update({key: values[key][i]})

    return G

In [156]:
graphs = []
for dataset in datasets:
    graphs.append(to_networkx(dataset))

In [179]:
number_of_nodes = []
clustering_coef=[]
density=[]
degree_assort=[]
pearson_assort = []
avg_degree_connectivity = []
for i,gr in enumerate(graphs):
            #число узлов 
        number_of_nodes.append(gr.number_of_nodes())
            #плотность
        density.append(nx.density(gr))
             #ассортативность
        degree_assort.append(nx.degree_assortativity_coefficient(gr))
        clustering_coef.append(np.mean(list(nx.clustering(gr).values())))
        pearson_assort.append(nx.degree_pearson_correlation_coefficient(gr))
        avg_degree_connectivity.append(np.mean(list(nx.average_degree_connectivity(gr).values())))

In [180]:
avg_degree_connectivity

[7.752167247488276,
 8.501729254240026,
 16.036628950519297,
 97.02011325261988,
 154.3084758098882,
 20.329004606257676,
 41.32114092667598,
 26.240967455468297,
 21.706734237314226,
 22.059571596457996,
 21.586676805729937,
 21.88705889180154,
 22.670815775281447,
 21.507048460428514,
 21.44778341093095,
 21.64140603201591,
 21.33800037871585,
 21.039414885529407]

In [181]:
df = pd.DataFrame({'Name of dataset' : datasets_names,
                                'Number of nodes (NN)' :number_of_nodes,
                                'Clustering coefficient (CC)': clustering_coef,'Density(D)':density, 'Degree assortativity coefficient(DAC)':degree_assort, 'Pearson degree assortativity coefficient (PDAC)': pearson_assort, 'Average Degree Connectivity (ADC)' : avg_degree_connectivity })
df.sort_values(by=['Average Degree Connectivity (ADC)'])

,Name of dataset,Number of nodes (NN),Clustering coefficient (CC),Density(D),Degree assortativity coefficient(DAC),Pearson degree assortativity coefficient (PDAC),Average Degree Connectivity (ADC)
0,Cora,2708,0.240673,0.001440,-0.065871,-0.065871,7.752167
1,Citeseer,3327,0.141471,0.000823,0.048378,0.048378,8.501729
2,Pubmed,19717,0.060175,0.000228,-0.043640,-0.043640,16.036629
5,CS,18333,0.342523,0.000487,0.112609,0.112609,20.329005
17,Homophily09,5000,0.057129,0.002384,-0.039710,-0.039710,21.039415
16,Homophily08,5000,0.040033,0.002384,-0.042703,-0.042703,21.338000
14,Homophily06,5000,0.023043,0.002384,-0.050115,-0.050115,21.447783
13,Homophily05,5000,0.018294,0.002384,-0.049194,-0.049194,21.507048
10,Homophily02,5000,0.011378,0.002384,-0.051903,-0.051903,21.586677
15,Homophily07,5000,0.030515,0.002384,-0.043096,-0.043096,21.641406


In [177]:
df.sort_values(by=['Pearson degree assortativity coefficient (PDAC)'])

,Name of dataset,Number of nodes (NN),Clustering coefficient (CC),Density(D),Degree assortativity coefficient(DAC),Pearson degree assortativity coefficient (PDAC),Average Degree Connectivity (ADC)
0,Cora,2708,0.240673,0.001440,-0.065871,-0.065871,21.039415
4,Computers,13752,0.344126,0.002600,-0.056488,-0.056488,21.039415
8,Homophily0,5000,0.009002,0.002384,-0.054907,-0.054907,21.039415
9,Homophily01,5000,0.011408,0.002384,-0.053471,-0.053471,21.039415
10,Homophily02,5000,0.011378,0.002384,-0.051903,-0.051903,21.039415
14,Homophily06,5000,0.023043,0.002384,-0.050115,-0.050115,21.039415
11,Homophily03,5000,0.012979,0.002384,-0.049402,-0.049402,21.039415
13,Homophily05,5000,0.018294,0.002384,-0.049194,-0.049194,21.039415
12,Homophily04,5000,0.014314,0.002384,-0.046596,-0.046596,21.039415
3,Photo,7650,0.403979,0.004070,-0.044943,-0.044943,21.039415


In [163]:
df.sort_values(by=['Degree assortativity coefficient(DAC)'])

,Name of dataset,Number of nodes (NN),Clustering coefficient (CC),Density(D),Degree assortativity coefficient(DAC)
0,Cora,2708,0.240673,0.001440,-0.065871
4,Computers,13752,0.344126,0.002600,-0.056488
8,Homophily0,5000,0.009002,0.002384,-0.054907
9,Homophily01,5000,0.011408,0.002384,-0.053471
10,Homophily02,5000,0.011378,0.002384,-0.051903
14,Homophily06,5000,0.023043,0.002384,-0.050115
11,Homophily03,5000,0.012979,0.002384,-0.049402
13,Homophily05,5000,0.018294,0.002384,-0.049194
12,Homophily04,5000,0.014314,0.002384,-0.046596
3,Photo,7650,0.403979,0.004070,-0.044943
